设置代理

In [1]:
import os
import requests

# 临时设置环境变量
os.environ['http_proxy'] = 'http://127.0.0.1:7890'
os.environ['https_proxy'] = 'http://127.0.0.1:7890'
os.environ['all_proxy'] = 'socks5://127.0.0.1:7890'

# 测试请求
response = requests.get('https://www.google.com')
print(response.status_code)

200


定义openai的Client

In [2]:
import time

class OpenaiClient:
    def __init__(self, keys=None, start_id=None, proxy=None):
        import openai
        from openai import OpenAI
        
        if isinstance(keys, str):
            keys = [keys]
        if keys is None:
            raise "Please provide OpenAI Key."

        self.key = keys
        self.key_id = start_id or 0
        self.key_id = self.key_id % len(self.key)
        self.api_key = self.key[self.key_id % len(self.key)]
        # 下面这一行base_url="https://api.gpts.vin/v1"是我自己加的
        # self.client = OpenAI(base_url="https://uiuiapi.com/v1", api_key=self.api_key)
        self.client = OpenAI(api_key=self.api_key)

    def chat(self, *args, return_text=False, reduce_length=False, **kwargs):
        while True:
            try:
                completion = self.client.chat.completions.create(*args, **kwargs, timeout=30)
                break
            except Exception as e:
                print(str(e))
                if "This model's maximum context length is" in str(e):
                    print('reduce_length')
                    return 'ERROR::reduce_length'
                time.sleep(0.1)
        if return_text:
            completion = completion.choices[0].message.content
        return completion

    def text(self, *args, return_text=False, reduce_length=False, **kwargs):
        while True:
            try:
                completion = self.client.completions.create(
                    *args, **kwargs
                )
                break
            except Exception as e:
                print(e)
                if "This model's maximum context length is" in str(e):
                    print('reduce_length')
                    return 'ERROR::reduce_length'
                time.sleep(0.1)
        if return_text:
            completion = completion.choices[0].text
        return completion

def run_llm(messages, api_key=None, model_name="gpt-3.5-turbo"):
    if 'gpt' in model_name:
        Client = OpenaiClient
    # elif 'o1' in model_name:
    #     Client = OpenaiClient_o1
    # elif 'claude' in model_name:
    #     Client = ClaudeClient
    # elif 'gemini' in model_name:
    #     Client = GeminiClient
    # elif 'moonshot' in model_name:
    #     Client = KimiClient
    # elif 'deepseek' in model_name:
    #     Client = BailianClient
    # else:
    #     Client = LitellmClient

    agent = Client(api_key)
    response = agent.chat(model=model_name, messages=messages, temperature=0, return_text=True) #temperature used to be 0
    # print(response)
    return response

数据集 导入举例
imdb:25000条
yelp:38000条
sst:2210条  
label:0 negative label:1 positive

数据集报错可能是缺文件fancyzhx/yelp/plain_text的train
stanfordnlp/imdb/train和unsupervised
stanfordnlp/sst-2/data/test和train


In [3]:
from datasets import load_dataset

# Load the dataset
# dataset = load_dataset("./stanfordnlp/imdb")
# dataset = load_dataset("./fancyzhx/yelp")
# dataset = load_dataset("./stanfordnlp/sst")
# test_dataset = dataset["test"]
# test_dataset[12500]["text"]
# print(test_dataset[12500]["label"])

# dataset = load_dataset("./stanfordnlp/imdb", split="test[:100]")  

定义字符级攻击函数

In [4]:
import random

def text_attack(text, 
                                          perturb_prob=1.0, 
                                          max_alterations=10, 
                                          max_per_word=3, 
                                          p_delete=0.33, 
                                          p_replace=0.33, 
                                          p_insert=0.34):
    """
    执行字符级扰动并返回统计信息（安全版本，避免死循环）
    - text：输入文本
    - perturb_prob：每个词被扰动的概率
    - max_alterations：全局最大扰动次数
    - max_per_word：每个词最多扰动次数
    - p_delete / p_replace / p_insert：三类扰动的概率分布（总和应为1.0）
    
    返回：
    - 扰动后的文本
    - 字典形式的扰动统计信息
    """
    assert abs(p_delete + p_replace + p_insert - 1.0) < 1e-5, "扰动概率之和必须为 1.0"

    words = text.split()
    attacked_words = []
    alterations = 0

    # 扰动类型统计
    count_delete, count_replace, count_insert = 0, 0, 0

    for word in words:
        if alterations >= max_alterations or random.random() > perturb_prob:
            attacked_words.append(word)
            continue

        # 特殊处理：只允许删除时避免词长为1死循环
        if len(word) <= 1 and p_delete == 1.0:
            attacked_words.append(word)
            continue

        word_list = list(word)
        local_alter = 0
        loop_counter = 0
        max_loops = 20  # 防止个别词进入死循环

        while alterations < max_alterations and local_alter < max_per_word and len(word_list) > 0:
            loop_counter += 1
            if loop_counter > max_loops:
                print(f"⚠️ 跳出 `{word}` 的扰动循环，避免死循环")
                break

            perturb_type = random.choices(
                population=["delete", "replace", "insert"],
                weights=[p_delete, p_replace, p_insert],
                k=1
            )[0]

            if perturb_type == "delete" and len(word_list) > 1:
                idx = random.randrange(len(word_list))
                word_list.pop(idx)
                alterations += 1
                local_alter += 1
                count_delete += 1

            elif perturb_type == "replace" and len(word_list) > 0:
                idx = random.randrange(len(word_list))
                word_list[idx] = random.choice('abcdefghijklmnopqrstuvwxyz')
                alterations += 1
                local_alter += 1
                count_replace += 1

            elif perturb_type == "insert":
                idx = random.randint(0, len(word_list))
                word_list.insert(idx, random.choice('abcdefghijklmnopqrstuvwxyz'))
                alterations += 1
                local_alter += 1
                count_insert += 1

        attacked_words.append("".join(word_list))

    stats = {
        "total_alterations": alterations,
        "delete": count_delete,
        "replace": count_replace,
        "insert": count_insert
    }

    return " ".join(attacked_words), stats

In [5]:
def create_messages(text):
    messages = [
        {'role': 'system',
         'content': "You are a helpful assistant specialized in sentiment analysis."},
        {'role': 'user',
         'content': f"Please classify the sentiment of the following review:\n\n\"{text}\"\nRespond **only** with 'positive' or 'negative'. Do not include any extra information."}
    ]
    return messages

随机数据集

In [ ]:
from datasets import load_dataset
import pickle
import random

# 加载数据集
# dataset_name = "./stanfordnlp/imdb"
# dataset_name = "./fancyzhx/yelp"
dataset_name = "./stanfordnlp/sst-2"
dataset = load_dataset(dataset_name, split="validation")
dataset = list(dataset)
dataset = random.sample(dataset, 278)

with open("./dataset_278/sst-2.pkl", "wb") as f:
    pickle.dump(dataset, f)

加载数据集、且预生成扰动文本，缓存下来:imdb, yelp

In [ ]:
attack_settings = [
    {"name": "delete", "p_delete": 1.0, "p_replace": 0.0, "p_insert": 0.0},
    {"name": "replace", "p_delete": 0.0, "p_replace": 1.0, "p_insert": 0.0},
    {"name": "insert", "p_delete": 0.0, "p_replace": 0.0, "p_insert": 1.0},
    {"name": "mix", "p_delete": 0.33, "p_replace": 0.34, "p_insert": 0.33},
]

# 设置随机种子，保证攻击文本一致
random.seed(42)

# 加载数据集
datasets = ["imdb", "yelp"]
# 从 pickle 文件中加载回来
for dataset_name in datasets:
    with open(f"./dataset_278/{dataset_name}.pkl", "rb") as f:
        dataset = pickle.load(f)
        # dataset = dataset[0:1]

    for attack_config in attack_settings:
        setting_name = attack_config["name"]
        p_delete = attack_config["p_delete"]
        p_replace = attack_config["p_replace"]
        p_insert = attack_config["p_insert"]

        # Step 1: 预生成扰动文本，缓存下来
        attacked_samples = []
        for item in dataset:
            original_text = item["text"]
            attacked_text = text_attack(
                original_text,
                perturb_prob=0.95,
                max_alterations=1000,
                max_per_word=2,
                p_delete=p_delete,
                p_replace=p_replace,
                p_insert=p_insert
            )
            attacked_samples.append({
                "label": item["label"],
                "original_text": original_text,
                "attacked_text": attacked_text
            })

        print(attacked_samples)
        print(len(attacked_samples))
        # 保存扰动样本为 pickle 文件
        save_path = f'./text_processed_278/{dataset_name}_{setting_name}.pkl'

        # 确保目标文件夹存在
        import os
        os.makedirs(os.path.dirname(save_path), exist_ok=True)

        with open(save_path, 'wb') as f:
            pickle.dump(attacked_samples, f)

        import pandas as pd
        # 保存为 CSV 文件
        df = pd.DataFrame(attacked_samples)
        df.to_csv(f'./text_processed_278/{dataset_name}_{setting_name}.csv', index=False)  # 保存为 CSV

        print(f"✅ 已保存：{save_path}")


加载数据集、且预生成扰动文本，缓存下来:sst-2

In [ ]:
import pickle
import random

attack_settings = [
    {"name": "delete", "p_delete": 1.0, "p_replace": 0.0, "p_insert": 0.0},
    {"name": "replace", "p_delete": 0.0, "p_replace": 1.0, "p_insert": 0.0},
    {"name": "insert", "p_delete": 0.0, "p_replace": 0.0, "p_insert": 1.0},
    {"name": "mix", "p_delete": 0.33, "p_replace": 0.34, "p_insert": 0.33},
]

# 设置随机种子，保证攻击文本一致
random.seed(42)

# 加载数据集
datasets = ["sst-2"]
# 从 pickle 文件中加载回来
for dataset_name in datasets:
    with open(f"./dataset_278/{dataset_name}.pkl", "rb") as f:
        dataset = pickle.load(f)
        # dataset = dataset[0:1]

    for attack_config in attack_settings:
        setting_name = attack_config["name"]
        p_delete = attack_config["p_delete"]
        p_replace = attack_config["p_replace"]
        p_insert = attack_config["p_insert"]

        # Step 1: 预生成扰动文本，缓存下来
        attacked_samples = []
        for item in dataset:
            original_text = item["sentence"]
            attacked_text = text_attack(
                original_text,
                perturb_prob=0.95,
                max_alterations=1000,
                max_per_word=2,
                p_delete=p_delete,
                p_replace=p_replace,
                p_insert=p_insert
            )
            attacked_samples.append({
                "label": item["label"],
                "original_text": original_text,
                "attacked_text": attacked_text
            })

        print(attacked_samples)
        print(len(attacked_samples))
        # 保存扰动样本为 pickle 文件
        save_path = f'./text_processed_278/{dataset_name}_{setting_name}.pkl'

        # 确保目标文件夹存在
        import os
        os.makedirs(os.path.dirname(save_path), exist_ok=True)

        with open(save_path, 'wb') as f:
            pickle.dump(attacked_samples, f)

        import pandas as pd
        # 保存为 CSV 文件
        df = pd.DataFrame(attacked_samples)
        df.to_csv(f'./text_processed_278/{dataset_name}_{setting_name}.csv', index=False)  # 保存为 CSV

        print(f"✅ 已保存：{save_path}")

LLM 判断

In [ ]:
# 已完成的组合
finished = {
    ('imdb', 'gpt-3.5-turbo', 'delete'),
    ('imdb', 'gpt-3.5-turbo', 'replace'),
    ('imdb', 'gpt-4-turbo', 'delete'),
    ('imdb', 'gpt-4o', 'delete'),
}

import openai
openai_key = 'sk-' # 补全
model_names = ['gpt-3.5-turbo', 'gpt-4-turbo','gpt-4o']
from tqdm import tqdm
import pickle
import pandas as pd

# 模型名 & 数据集 & 攻击模式
model_names = ['gpt-3.5-turbo', 'gpt-4-turbo', 'gpt-4o']
datasets = ['imdb', 'yelp', 'sst-2']
attack_modes = ['delete', 'replace', 'insert', 'mix']

# 输出路径
output_dir = './result_278'
os.makedirs(output_dir, exist_ok=True)

for dataset_name in datasets:
    for attack_mode in attack_modes:
        # 加载攻击样本
        file_path = f'./text_processed_278/{dataset_name}_{attack_mode}.pkl'
        with open(file_path, 'rb') as f:
            attacked_samples = pickle.load(f)
        
        for model_name in model_names:
            if (dataset_name, model_name, attack_mode) in finished:
                print(f"⏭️ 已完成，跳过：{dataset_name} | {model_name} | {attack_mode}")
                continue
            results = []

            for i, sample in enumerate(tqdm(attacked_samples, desc=f'{dataset_name} | {attack_mode} | {model_name}')):
                original_text = sample['original_text']
                attacked_text = sample['attacked_text']
                label = sample['label']

                messages_original = create_messages(original_text)
                messages_attacked = create_messages(attacked_text)

                result_original = run_llm(messages_original, api_key=openai_key, model_name=model_name)
                result_attacked = run_llm(messages_attacked, api_key=openai_key, model_name=model_name)

                results.append({
                    "id": i,
                    "label": label,
                    "result_original": result_original,
                    "result_attacked": result_attacked,
                    "original_text": original_text,
                    "attacked_text": attacked_text
                })

            # 保存为 CSV & PKL
            df = pd.DataFrame(results)
            df.to_csv(f'{output_dir}/{dataset_name}_{model_name}_{attack_mode}.csv', index=False)
            df.to_pickle(f'{output_dir}/{dataset_name}_{model_name}_{attack_mode}.pkl')
            print(f"✅ 保存成功: {dataset_name} | {model_name} | {attack_mode}")

改perturb_prob和max_alterations

In [13]:
import pickle

attack_settings = [
    {"name": "mix", "p_delete": 0.33, "p_replace": 0.34, "p_insert": 0.33},
]
perturb_prob = 0.15

# 设置随机种子，保证攻击文本一致
random.seed(42)

# 加载数据集
datasets = ["imdb", "yelp"]
# datasets = ["sst-2"]
# 从 pickle 文件中加载回来
for dataset_name in datasets:
    with open(f"./dataset_278/{dataset_name}.pkl", "rb") as f:
        dataset = pickle.load(f)

    for attack_config in attack_settings:
        setting_name = attack_config["name"]
        p_delete = attack_config["p_delete"]
        p_replace = attack_config["p_replace"]
        p_insert = attack_config["p_insert"]

        # Step 1: 预生成扰动文本，缓存下来
        attacked_samples = []
        for item in dataset:
            original_text = item["text"]
            # original_text = item["sentence"]
            attacked_text = text_attack(
                original_text,
                perturb_prob=perturb_prob,
                max_alterations=1000,
                max_per_word=2,
                p_delete=p_delete,
                p_replace=p_replace,
                p_insert=p_insert
            )
            attacked_samples.append({
                "label": item["label"],
                "original_text": original_text,
                "attacked_text": attacked_text
            })

        print(attacked_samples)
        print(len(attacked_samples))
        # 保存扰动样本为 pickle 文件
        save_path = f'./ablation_278/{dataset_name}_{setting_name}_{perturb_prob}.pkl'

        # 确保目标文件夹存在
        import os
        os.makedirs(os.path.dirname(save_path), exist_ok=True)

        with open(save_path, 'wb') as f:
            pickle.dump(attacked_samples, f)

        import pandas as pd
        # 保存为 CSV 文件
        df = pd.DataFrame(attacked_samples)
        df.to_csv(f'./ablation_278/{dataset_name}_{setting_name}_{perturb_prob}.csv', index=False)  # 保存为 CSV

        print(f"✅ 已保存：{save_path}")


[{'label': 1, 'original_text': "I just saw this wonderfully filmed movie that captures the essence of<br /><br />high-brow NYC, or any big city of mid-century America. The colors, the<br /><br />cars, the clothes and the coming of the Womens Movement. It reflects the<br /><br />comf-cozy attitudes of relationships between men and women in the<br /><br />corporate world. In some ways, things gave changed and in others, they<br /><br />haven't changed at all. Women still want what men have today, but they<br /><br />now have all sorts of laws and equality mandates to get it for them. In<br /><br />my opinion, beautiful women will still THROW themselves at men in<br /><br />pursuit of thier goals! The laws we have now against harrassment and<br /><br />all, were passed by unattractive women who wanted an equal chance to<br /><br />compete with prettier women who might be getting the positions soley<br /><br />based on thier looks and puting out! The real competition isn't between<br /><br

In [15]:
# # 已完成的组合
# finished = {
#     ('imdb', 'gpt-3.5-turbo', 'delete'),
#     ('imdb', 'gpt-3.5-turbo', 'replace'),
#     ('imdb', 'gpt-4-turbo', 'delete'),
#     ('imdb', 'gpt-4o', 'delete'),
# }

import openai
openai_key = 'sk-' # 补全
model_names = ['gpt-4o']
perturb_probs = ['0.75', '0.55', '0.35', '0.15']
from tqdm import tqdm
import pickle
import pandas as pd

# 模型名 & 数据集 & 攻击模式
model_names = ['gpt-4o']
datasets = ['imdb', 'yelp', 'sst-2']
attack_mode = 'mix'

# 输出路径
output_dir = './result_ablation_278'
os.makedirs(output_dir, exist_ok=True)

for dataset_name in datasets:
    for perturb_prob in perturb_probs:
        # 加载攻击样本
        file_path = f'./ablation_278/{dataset_name}_{attack_mode}_{perturb_prob}.pkl'
        with open(file_path, 'rb') as f:
            attacked_samples = pickle.load(f)
        
        for model_name in model_names:
            # if (dataset_name, model_name, attack_mode) in finished:
            #     print(f"⏭️ 已完成，跳过：{dataset_name} | {model_name} | {attack_mode}")
            #     continue
            results = []

            for i, sample in enumerate(tqdm(attacked_samples, desc=f'{dataset_name} | {attack_mode} | {model_name}')):
                original_text = sample['original_text']
                attacked_text = sample['attacked_text']
                label = sample['label']

                # messages_original = create_messages(original_text)
                messages_attacked = create_messages(attacked_text)

                # result_original = run_llm(messages_original, api_key=openai_key, model_name=model_name)
                result_attacked = run_llm(messages_attacked, api_key=openai_key, model_name=model_name)

                results.append({
                    "id": i,
                    "label": label,
                    "result_original": 'done in other file',
                    "result_attacked": result_attacked,
                    "original_text": original_text,
                    "attacked_text": attacked_text
                })

            # 保存为 CSV & PKL
            df = pd.DataFrame(results)
            df.to_csv(f'{output_dir}/{dataset_name}_{model_name}_{attack_mode}_{perturb_prob}.csv', index=False)
            df.to_pickle(f'{output_dir}/{dataset_name}_{model_name}_{attack_mode}_{perturb_prob}.pkl')
            print(f"✅ 保存成功: {dataset_name} | {model_name} | {attack_mode} | {perturb_prob} ")

imdb | mix | gpt-4o: 100%|██████████| 278/278 [09:03<00:00,  1.96s/it]


✅ 保存成功: imdb | gpt-4o | mix | 0.75 


imdb | mix | gpt-4o: 100%|██████████| 278/278 [08:58<00:00,  1.94s/it]


✅ 保存成功: imdb | gpt-4o | mix | 0.55 


imdb | mix | gpt-4o: 100%|██████████| 278/278 [09:02<00:00,  1.95s/it]


...

✅ 保存成功: sst-2 | gpt-4o | mix | 0.55 


sst-2 | mix | gpt-4o: 100%|██████████| 278/278 [09:26<00:00,  2.04s/it]


✅ 保存成功: sst-2 | gpt-4o | mix | 0.35 


sst-2 | mix | gpt-4o: 100%|██████████| 278/278 [09:14<00:00,  1.99s/it]

✅ 保存成功: sst-2 | gpt-4o | mix | 0.15 
